In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import torch

import re

# from sklearn.model_selection import KFold
# from sklearn.metrics import accuracy_score
# from sklearn.neighbors import KNeighborsClassifier

- Load your embeddings and labels (assuming a DataFrame format)
- df should have columns 'image1', 'image2' for image identifiers, and 'label' for same/different person
- embeddings = {'img_id1': tensor1, 'img_id2': tensor2, ...}

In [8]:
file_path = 'formated_pairs.txt'

with open(file_path, 'r') as file:
    content = file.read()
    print(content)

Abel_Pacheco_0.alias Abel_Pacheco_3.alias
Akhmed_Zakayev_0.alias Akhmed_Zakayev_2.alias
Akhmed_Zakayev_1.alias Akhmed_Zakayev_2.alias
Amber_Tamblyn_0.alias Amber_Tamblyn_1.alias
Anders_Fogh_Rasmussen_0.alias Anders_Fogh_Rasmussen_2.alias
Anders_Fogh_Rasmussen_0.alias Anders_Fogh_Rasmussen_3.alias
Angela_Bassett_0.alias Angela_Bassett_4.alias
Angela_Bassett_1.alias Angela_Bassett_4.alias
Angela_Bassett_2.alias Angela_Bassett_3.alias
Ann_Veneman_2.alias Ann_Veneman_4.alias
Ann_Veneman_5.alias Ann_Veneman_9.alias
Ann_Veneman_9.alias Ann_Veneman_10.alias
Anthony_Fauci_0.alias Anthony_Fauci_1.alias
Antony_Leung_0.alias Antony_Leung_1.alias
Antony_Leung_1.alias Antony_Leung_2.alias
Anwar_Ibrahim_0.alias Anwar_Ibrahim_1.alias
Augusto_Pinochet_0.alias Augusto_Pinochet_1.alias
Barbara_Brezigar_0.alias Barbara_Brezigar_1.alias
Benjamin_Netanyahu_0.alias Benjamin_Netanyahu_3.alias
Benjamin_Netanyahu_3.alias Benjamin_Netanyahu_4.alias
Bernard_Law_1.alias Bernard_Law_2.alias
Bernard_Law_2.alias Ber

In [9]:
embeddings = torch.load('embeddings_dict.pt')
print(embeddings.keys())

dict_keys(['Chris_Moore/Chris_Moore_0001.torch', 'Robert_Towne/Robert_Towne_0001.torch', 'Andy_Warhol/Andy_Warhol_0001.torch', 'Tim_Conway/Tim_Conway_0002.torch', 'Tim_Conway/Tim_Conway_0001.torch', 'Tim_Conway/Tim_Conway_0003.torch', 'Bill_Byrne/Bill_Byrne_0001.torch', 'Shaun_Rusling/Shaun_Rusling_0001.torch', 'Giuseppe_Morchio/Giuseppe_Morchio_0001.torch', 'David_Beckham/David_Beckham_0027.torch', 'David_Beckham/David_Beckham_0009.torch', 'David_Beckham/David_Beckham_0001.torch', 'David_Beckham/David_Beckham_0020.torch', 'David_Beckham/David_Beckham_0031.torch', 'David_Beckham/David_Beckham_0016.torch', 'David_Beckham/David_Beckham_0024.torch', 'David_Beckham/David_Beckham_0022.torch', 'David_Beckham/David_Beckham_0015.torch', 'David_Beckham/David_Beckham_0011.torch', 'David_Beckham/David_Beckham_0029.torch', 'David_Beckham/David_Beckham_0013.torch', 'Steven_Hatfill/Steven_Hatfill_0001.torch', 'Steven_Hatfill/Steven_Hatfill_0002.torch', 'Wang_Hailan/Wang_Hailan_0001.torch', 'Alfonso_

In [10]:
keys = list(embeddings.keys())
first_key = keys[0]
first_embedding = embeddings[first_key]
first_shape = first_embedding.shape
print("First shape:", first_shape)

First shape: torch.Size([1, 512])


In [11]:
print(first_embedding)

tensor([[-2.9780e-02,  3.7657e-02,  2.1946e-02, -1.9594e-02,  3.7291e-02,
         -6.2375e-02,  1.5618e-02,  2.6089e-02,  1.6782e-03,  4.8763e-02,
         -1.2171e-02, -8.3226e-02,  7.6907e-02,  4.8538e-02, -2.9783e-02,
          4.6696e-03,  9.5117e-02, -2.1650e-02,  1.5227e-02, -2.5513e-02,
          2.4792e-02,  1.0168e-01, -2.4293e-02,  1.4754e-03,  1.0009e-01,
          2.0040e-02, -4.6910e-02,  4.4188e-02,  1.3272e-05,  1.3830e-03,
         -5.4306e-02, -3.6621e-02,  7.7124e-03,  1.3960e-02, -8.6398e-02,
         -1.6787e-02,  2.3938e-02, -5.6595e-02, -1.5849e-03,  4.8307e-02,
          9.2158e-02, -5.5366e-02, -4.5653e-02,  5.7736e-02,  6.6813e-02,
          3.0021e-03,  1.9527e-02, -2.3784e-02, -3.6341e-02,  1.9920e-02,
          2.8588e-02, -7.9920e-02,  3.2237e-02,  4.4983e-02, -7.4097e-02,
          1.8353e-03, -1.9306e-02, -6.9068e-03, -1.1770e-02,  3.1334e-02,
          8.6310e-03,  7.2572e-03, -9.2259e-02,  1.0867e-03, -8.4678e-02,
         -5.4547e-03, -5.1701e-02, -3.

In [12]:
pairs = []
with open('formated_pairs.txt', 'r') as f:
    for line in f:
        image1, image2 = line.strip().split()  
        pairs.append((image1, image2))
pairs[0]

('Abel_Pacheco_0.alias', 'Abel_Pacheco_3.alias')

In [13]:
def rename_keys(dictionary):
    new_dictionary = {}
    for key in dictionary:
        match = re.match(r'(.+)/.+_(\d+)\.torch', key)
        if match:
            name = match.group(1)
            index = str(int(match.group(2)) - 1) 
            new_key = f"{name}_{index}" 
            new_dictionary[new_key] = dictionary[key] 
    return new_dictionary

In [14]:
embeddings_final = rename_keys(embeddings)
print(embeddings_final.keys())

dict_keys(['Chris_Moore_0', 'Robert_Towne_0', 'Andy_Warhol_0', 'Tim_Conway_1', 'Tim_Conway_0', 'Tim_Conway_2', 'Bill_Byrne_0', 'Shaun_Rusling_0', 'Giuseppe_Morchio_0', 'David_Beckham_26', 'David_Beckham_8', 'David_Beckham_0', 'David_Beckham_19', 'David_Beckham_30', 'David_Beckham_15', 'David_Beckham_23', 'David_Beckham_21', 'David_Beckham_14', 'David_Beckham_10', 'David_Beckham_28', 'David_Beckham_12', 'Steven_Hatfill_0', 'Steven_Hatfill_1', 'Wang_Hailan_0', 'Alfonso_Cuaron_0', 'Li_Peng_5', 'Li_Peng_0', 'Li_Peng_8', 'Li_Peng_6', 'Li_Peng_7', 'Li_Peng_1', 'Li_Peng_3', 'Muammar_Gaddafi_1', 'Muammar_Gaddafi_0', 'Hartmut_Mehdorn_0', 'Jamie_Martin_0', 'Sterling_Hitchcock_0', 'Colin_Montgomerie_1', 'Colin_Montgomerie_3', 'Colin_Montgomerie_0', 'Colin_Montgomerie_2', 'Colin_Montgomerie_4', 'Frank_Zappa_0', 'Alan_Dreher_0', 'Serge_Melac_0', 'Vladimiro_Montesinos_2', 'Vladimiro_Montesinos_1', 'Vladimiro_Montesinos_0', 'Diego_Diego_Lerman_0', 'Ajit_Agarkar_0', 'Junichiro_Koizumi_25', 'Junichiro_

In [15]:
def remove_alias(pairs):
    return [(nom1.replace('.alias', ''), nom2.replace('.alias', '')) for nom1, nom2 in pairs]

In [16]:
pairs_names = remove_alias(pairs)
pairs_names[0]

('Abel_Pacheco_0', 'Abel_Pacheco_3')

In [17]:
df = pd.DataFrame(pairs_names, columns=['Nom1', 'Nom2'])
df

,Nom1,Nom2
0,Abel_Pacheco_0,Abel_Pacheco_3
1,Akhmed_Zakayev_0,Akhmed_Zakayev_2
2,Akhmed_Zakayev_1,Akhmed_Zakayev_2
3,Amber_Tamblyn_0,Amber_Tamblyn_1
4,Anders_Fogh_Rasmussen_0,Anders_Fogh_Rasmussen_2
...,...,...
5995,Scott_Wolf_1,Troy_Polamalu_0
5996,Sergei_Alexandrovitch_Ordzhonikidze_0,Yolanda_King_0
5997,Shane_Loux_0,Val_Ackerman_0
5998,Shawn_Marion_0,Shirley_Jones_0


In [18]:
def remove_index(nom):
    return '_'.join(nom.split('_')[:-1])

In [19]:
for i in df.index:
  nom1 = remove_index(df.loc[i,'Nom1'])
  nom2 = remove_index(df.loc[i,'Nom2'])
  if nom1==nom2:
    df.loc[i,'label']=1
  else:
    df.loc[i,'label']=0

In [20]:
df

,Nom1,Nom2,label
0,Abel_Pacheco_0,Abel_Pacheco_3,1.0
1,Akhmed_Zakayev_0,Akhmed_Zakayev_2,1.0
2,Akhmed_Zakayev_1,Akhmed_Zakayev_2,1.0
3,Amber_Tamblyn_0,Amber_Tamblyn_1,1.0
4,Anders_Fogh_Rasmussen_0,Anders_Fogh_Rasmussen_2,1.0
...,...,...,...
5995,Scott_Wolf_1,Troy_Polamalu_0,0.0
5996,Sergei_Alexandrovitch_Ordzhonikidze_0,Yolanda_King_0,0.0
5997,Shane_Loux_0,Val_Ackerman_0,0.0
5998,Shawn_Marion_0,Shirley_Jones_0,0.0


In [21]:
distances = []
for nom1, nom2 in pairs_names:
  distance = torch.dist(embeddings_final[nom1], embeddings_final[nom2], p=2)
  distances.append(distance)
print(distances)

[tensor(0.9318), tensor(0.6290), tensor(0.7778), tensor(0.8051), tensor(0.8463), tensor(0.9795), tensor(0.8205), tensor(0.9586), tensor(0.9281), tensor(0.6955), tensor(0.7964), tensor(0.9739), tensor(0.6996), tensor(0.8895), tensor(0.9424), tensor(0.9473), tensor(0.5991), tensor(0.9471), tensor(0.8115), tensor(0.8671), tensor(1.0123), tensor(0.8716), tensor(0.8316), tensor(0.8759), tensor(1.0474), tensor(0.9499), tensor(0.9318), tensor(0.9652), tensor(1.0024), tensor(1.0166), tensor(0.9798), tensor(0.8237), tensor(1.0176), tensor(0.7410), tensor(0.9247), tensor(0.9505), tensor(0.9283), tensor(0.9580), tensor(0.8478), tensor(0.9453), tensor(0.9471), tensor(1.0007), tensor(0.7124), tensor(0.8985), tensor(1.0221), tensor(0.9060), tensor(1.0325), tensor(1.1239), tensor(0.7490), tensor(0.8407), tensor(1.1957), tensor(0.9651), tensor(0.9910), tensor(1.0536), tensor(0.9124), tensor(0.8541), tensor(0.8292), tensor(0.5565), tensor(1.0024), tensor(1.0482), tensor(0.6996), tensor(0.9786), tensor(

In [22]:
df['distances'] = distances
df

,Nom1,Nom2,label,distances
0,Abel_Pacheco_0,Abel_Pacheco_3,1.0,tensor(0.9318)
1,Akhmed_Zakayev_0,Akhmed_Zakayev_2,1.0,tensor(0.6290)
2,Akhmed_Zakayev_1,Akhmed_Zakayev_2,1.0,tensor(0.7778)
3,Amber_Tamblyn_0,Amber_Tamblyn_1,1.0,tensor(0.8051)
4,Anders_Fogh_Rasmussen_0,Anders_Fogh_Rasmussen_2,1.0,tensor(0.8463)
...,...,...,...,...
5995,Scott_Wolf_1,Troy_Polamalu_0,0.0,tensor(1.4076)
5996,Sergei_Alexandrovitch_Ordzhonikidze_0,Yolanda_King_0,0.0,tensor(1.4186)
5997,Shane_Loux_0,Val_Ackerman_0,0.0,tensor(1.4423)
5998,Shawn_Marion_0,Shirley_Jones_0,0.0,tensor(1.4111)


# SVM

In [28]:
X = df[['distances']]
y = df['label']

In [29]:
# Split the dataset into a training set and a test set

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [31]:
# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [32]:
# SVM Model training
svm_model = SVC(kernel='rbf', C=1.0, gamma='auto')
svm_model.fit(X_train_scaled, y_train)

SVC(gamma='auto')

In [33]:
# Making predictions
y_pred = svm_model.predict(X_test_scaled)

In [34]:
# Evaluating the model
print("Accuracy on Test Set:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy on Test Set: 0.99
Classification Report:
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99       722
         1.0       0.99      0.99      0.99       778

    accuracy                           0.99      1500
   macro avg       0.99      0.99      0.99      1500
weighted avg       0.99      0.99      0.99      1500



In [ ]:
# i just applied an svm model to my classification problem that have the 

In [ ]:
# def calculate_euclidean_distance(embeddings, img1, img2):
#     """Calculate the Euclidean distance between two embeddings."""
#     emb1 = embeddings[img1]
#     emb2 = embeddings[img2]
#     return torch.norm(emb1 - emb2, p=2).item()

# # Calculate distances and prepare the DataFrame
# df['distance'] = df.apply(lambda row: calculate_euclidean_distance(embeddings, row['image1'], row['image2']), axis=1)

# # Features and labels
# X = df[['distance']].values
# y = df['label'].values

# # Split the dataset into a training set and a test set
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# # Feature scaling
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# SVM Model training
svm_model = SVC(kernel='rbf', C=1.0, gamma='auto')
svm_model.fit(X_train_scaled, y_train)

# # Making predictions
# y_pred = svm_model.predict(X_test_scaled)

# Evaluating the model
print("Accuracy on Test Set:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))